In [34]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')

In [35]:
slownik = pd.read_table('polimorfologik-2.1.txt', sep = ";", header = None, names = ['lemma', 'term', 'POS'])

przyslowia = pd.read_table('przyslowia.txt', encoding = 'utf-8', sep = ',')

In [36]:
przyslowia.head()

,"Jesień tego nie zrodzi, czego wiosna nie zasiała",Czego wiosna nie zasiała – jesień nie urodzi,"Św. Bartłomiej pogodny, jesień pogodna","Bartłomiej zwiastuje, jaka jesień następuje, i czy w przyszłym latku dożyjesz dostatku","Bartłomiej zwiastuje, jaka jesień następuje","Bartłomieja cały wrzesień naśladuje, i z nim jesień","Bartłomieja świętego dzień w jakiej zostaje porze, taką jesień bez ochyby daje","Jaki Bartek niesie dzień, taka będzie i jesień","Jaki Bartek niesie dzień, taka będzie i jesień.1","Jaki Bartek, taki wrzesień, jaki Marcin, taka zima"


Oczyszczenie przysłów

In [37]:
def cleaned_text(saying):
    # normalizacja tekstu
    myslnik = re.compile("["u"\u2013""]+", flags = re.UNICODE)
    temp = re.sub(myslnik, "", saying)
    temp = re.sub("\s{2,}", " ", temp) #usunięcie podwójnych spacji
    temp = re.sub("(\r\n|\r|\n)", " ", temp) #usunięcie przerw między słowami, entery
    temp = temp.lower() #zamienia wszystkie znaki na małe litery
    #temp = re.sub("rt", "", temp) #usunięcie retweetów
    #temp = re.sub("&amp", "", temp) #usunięcie kodu htmlowego
    #temp = re.sub("#[a-z,A-Z]*", "", temp) #usunięcie hasztagów
    #temp = re.sub("@\w+", "", temp) #usunięcie wzmianek, oznaczeń, tagowań
    #temp = re.sub("(f|ht)(tp)([^ ]*)", "", temp) #usunięcie urlów
    #temp = re.sub("http(s?)([^ ]*)", "", temp) #usunięcie urlów
    temp = re.sub("[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", " ", temp) #usunięcie znaków specjalnych
    temp = re.sub("\d", "", temp) #usunięcie cyfr
    temp = re.sub("\s{2,}", " ", temp) #ponowne usunięcie podwójnych spacji
    temp = re.sub("[^\w\s]", "", temp) #usunięcie znaków specjalnych
    temp = temp.strip() #usunięcie spacji lub odstępów na końcach tekstu
    
    #tokenizacja
    cleaned_and_tokenized = nltk.word_tokenize(temp)
    
    return cleaned_and_tokenized

Oczyszczone przysłowia po tokenizacji

In [38]:
cleaned_sayings = [cleaned_text(saying) for saying in przyslowia]
print(cleaned_sayings)

[['jesień', 'tego', 'nie', 'zrodzi', 'czego', 'wiosna', 'nie', 'zasiała'], ['czego', 'wiosna', 'nie', 'zasiała', 'jesień', 'nie', 'urodzi'], ['św', 'bartłomiej', 'pogodny', 'jesień', 'pogodna'], ['bartłomiej', 'zwiastuje', 'jaka', 'jesień', 'następuje', 'i', 'czy', 'w', 'przyszłym', 'latku', 'dożyjesz', 'dostatku'], ['bartłomiej', 'zwiastuje', 'jaka', 'jesień', 'następuje'], ['bartłomieja', 'cały', 'wrzesień', 'naśladuje', 'i', 'z', 'nim', 'jesień'], ['bartłomieja', 'świętego', 'dzień', 'w', 'jakiej', 'zostaje', 'porze', 'taką', 'jesień', 'bez', 'ochyby', 'daje'], ['jaki', 'bartek', 'niesie', 'dzień', 'taka', 'będzie', 'i', 'jesień'], ['jaki', 'bartek', 'niesie', 'dzień', 'taka', 'będzie', 'i', 'jesień'], ['jaki', 'bartek', 'taki', 'wrzesień', 'jaki', 'marcin', 'taka', 'zima']]


Selekcja kolumn i normalizacja słów w słowniku

In [39]:
slownik_wyb = slownik.loc[:, 'lemma':'term']
slownik_m = slownik_wyb.apply(lambda x: x.str.lower(), axis = 0)

slownik_m.head()

,lemma,term
0,aa,aa
1,aap,aap
2,abb,abb
3,abba,abb
4,abba,abb-ach


Usunięcie duplikatów słów odmienionych

In [40]:
#slownik_m[slownik_m['termy'] == 'kotki']

slownik_bez = slownik_m.drop_duplicates(subset = 'term')
slownik_bez.head()

,lemma,term
0,aa,aa
1,aap,aap
2,abb,abb
4,abba,abb-ach
5,abba,abb-ami


Lematyzacja

In [41]:
slownik = dict(zip(slownik_bez['term'], slownik_bez['lemma']))

In [42]:
def lemmatized_sayings(sayings, slownik):
    """
    Funkcja lematyzuje przysłowia z listy 'sayings' z wykorzystaniem słownika 'slownik'
    """
    lemmatized_sayings = []  #lista dla lematyzowanych przysłów

    for saying in sayings:
        lemmatized_words = []  #lista dla lematyzowanych słów
        for word in saying:
            lemma = slownik.get(word.lower(), word)  #pobiera lemat ze słownika lub zostawia oryginalne słowo, jeśli nie ma lematu
            lemmatized_words.append(lemma)  #dodaje lematyzowane słowo do listy
        lemmatized_saying = " ".join(lemmatized_words)  #łączy lematyzowane słowa w całe przysłowie
        lemmatized_sayings.append(lemmatized_saying)  #dodaje lematyzowane przysłowie do listy

    return lemmatized_sayings

In [43]:
lemmatized_sayings = lemmatized_sayings(cleaned_sayings, slownik)

In [44]:
print(lemmatized_sayings)

['jesień ten nie zrodzić co wiosna nie zasiać', 'co wiosna nie zasiać jesień nie urodzić', 'święty bartłomiej pogodny jesień pogodno', 'bartłomiej zwiastować jak jesień następować i czy wat przyszły latek dożyć dostatek', 'bartłomiej zwiastować jak jesień następować', 'bartłomiej cała wrzesień naśladować i z nim jesień', 'bartłomiej święte dzienia wat jaki zostawać por taka jesień bez ochyba dawać', 'jacques bartek nieść dzienia taka być i jesień', 'jacques bartek nieść dzienia taka być i jesień', 'jacques bartek taka wrzesień jacques marcin taka zima']
